In [10]:
open System

type Connections = { SortedJoins : ((int*int) * (int*int)) list; Start : (int*int) option }

module Connections = 
    let empty = { SortedJoins = List.empty; Start = None }

    let add (end1 : int*int, end2 : int*int) connections =
        let a = min end1 end2
        let b = max end1 end2
        { connections with SortedJoins = (a,b)::connections.SortedJoins }

    let getConnections xy connections =
        connections.SortedJoins
        |> List.filter (fun (a,b) -> a=xy || b=xy)

    let getDestinations xy connections =
        connections.SortedJoins
        |> List.choose (
            fun (a,b) -> 
                if a=xy then 
                    Some b
                elif b=xy then
                    Some a
                else
                    None
        )

module PipeMap =
    let addPiece connections ((x, y), piece) =
        let here = (x,y)
        let west = (x-1,y)
        let east = (x+1,y)
        let north = (x,y-1)
        let south = (x,y+1)
        match piece with
        | '.' -> 
            connections
        | '|' ->
            connections
            |> Connections.add (north,here)
            |> Connections.add (here,south)
        | '-' ->
            connections
            |> Connections.add (west,here)
            |> Connections.add (here,east)
        | 'L' ->
            connections
            |> Connections.add (north,here)
            |> Connections.add (here,east)
        | 'J' ->
            connections
            |> Connections.add (north,here)
            |> Connections.add (here,west)
        | '7' ->
            connections
            |> Connections.add (here,west)
            |> Connections.add (south,here)
        | 'F' ->
            connections
            |> Connections.add (here,east)
            |> Connections.add (south,here)
        | 'S' ->
            { connections with Start = Some here }
        | _ ->
            failwithf "Unknown piece: %c" piece
    
    let filterMismatched connections =
        let connected = 
            connections.SortedJoins
            |> List.countBy id
            |> List.filter (fun (_,count) -> count = 2)
            |> List.map fst
        
        let fromStart =
            match connections.Start with
            | None -> []
            | Some start -> connections |> Connections.getConnections start
        
        { connections with SortedJoins = connected @ fromStart }
    
    let getSteps connections =
        let rec nextStep xy visited =
            let prevStep = visited |> List.tryHead
            if Some xy = connections.Start && not (prevStep = None) then
                xy::visited
            else
                let destinations = 
                    connections 
                    |> Connections.getDestinations xy
                    |> List.filter (fun next -> not (Some next = prevStep))

                match destinations with
                | [] -> failwithf "No destinations for %A" xy
                | next::_ -> nextStep next (xy::visited) 

        match connections.Start with
        | None -> []
        | Some start -> nextStep start []

    let parseInput lines =
        let height = Array.length lines
        let width = Seq.length lines[0]

        let chars = 
            lines
            |> Seq.indexed
            |> Seq.collect (
                fun (y, line) ->
                    line
                    |> Seq.mapi (
                        fun x piece ->
                            (x,y), piece
                    )
            )

        
        chars
        |> Seq.fold addPiece Connections.empty 
        |> filterMismatched

let maxDistance connections =
    let steps = PipeMap.getSteps connections
    (steps |> List.length) / 2


In [12]:
#r "nuget:FsUnit.xUnit"

open FsUnitTyped

let testInput =
    [|
        "..F7."
        ".FJ|."
        "SJ.L7"
        "|F--J"
        "LJ..."
    |]

let testBoard = PipeMap.parseInput testInput
maxDistance testBoard |> shouldEqual 8


Installed Packages FsUnit.xUnit, 5.6.1